# Pythonで学ぶ有限要素法（PINNs編）
線形等方弾性体の変形をPINNs(Physics-informed neural network)により解くサンプルコードです．  
YouTubeチャンネル [大学生・技術者のための有限要素法](https://www.youtube.com/@fempython) の動画に対応しています．  
このnotebookには数式と対応するPythonスクリプトしか記載していないため，詳細な解説は対応する動画をご参照ください．

## 問3. PINNs(Physics-informed neural network)

運動方程式
$$
\frac{ \partial \sigma^{ij}}{ \partial x^j} + b^i = 0
$$
構成式
$$
\sigma^{ij} = C^{ijkl} \varepsilon_{kl}
$$
Neumann境界条件
$$
t^i = \bar{t}{}^i, \quad t^i = \sigma^{ij}n_j
$$
Dirichlet境界条件
$$
u_i = \bar{u}_i
$$
全損失
$$
L
=\alpha_{\mathrm{EoM}}L_{\mathrm{EoM}}+\alpha_{\mathrm{CL}}L_{\mathrm{CL}} +\alpha_{\mathrm{NBC}}L_{\mathrm{NBC}}  +\alpha_{\mathrm{DBC}}L_{\mathrm{DBC}}
$$
運動方程式に関する損失
$$
L_{\mathrm{EoM}}
=\left\| \frac{\partial \sigma^{ij}}{\partial x^j}+b^i \right\|^2_2
$$
構成式に関する損失
$$
L_{\mathrm{CL}}
= \left\| \sigma^{ij}
- \frac{1}{2}C^{ijkl} \varepsilon_{ij} \right\|^2_2
$$
Neumann境界条件に関する誤差
$$
L_{\mathrm{NBC}}=\left\| \bar{t}{}^{\,i} - \sigma^{ij}n_{j} \right\|^2_2
$$
Dirichlet境界条件に関する誤差
$$
L_{\mathrm{DBC}}=\left\| \bar{u}{}_i - u_i \right\|^2_2
$$



## PINNsの実装

In [ ]:
%%capture --no-stderr
%pip install -U pyDOE

In [ ]:
import numpy as np
import torch
import random

# GPUが利用できる場合は利用
device = "cuda" if torch.cuda.is_available() else "cpu"

# 乱数シードの設定
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(1234)

### ニューラルネットワークの設定

In [ ]:
import torch.nn as nn
import torch.nn.init as init

In [ ]:
class Net(nn.Module):
    def __init__(
        self,
        n_inputs: int = 2,
        n_outputs: int = 5,
        n_hidden_layers: int = 2,
        n_hidden_neurons: int = 5,
        gain: float = 0.5,
    ):
        super().__init__()
        # 入力層
        self.input_layer = nn.Linear(n_inputs, n_hidden_neurons)
        # 隠れ層
        self.hidden_layers = nn.ModuleList(
            [
                nn.Linear(n_hidden_neurons, n_hidden_neurons)
                for _ in range(n_hidden_layers)
            ]
        )
        # 出力層
        self.output_layer = nn.Linear(n_hidden_neurons, n_outputs)
        # 活性化関数
        self.activation = nn.Tanh()

        self.gain = gain
        # 重みの初期化
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            # Xavier正規分布による初期化
            init.xavier_normal_(module.weight, gain=self.gain)
            if module.bias is not None:
                init.zeros_(module.bias)

    def forward(self, x):
        x = self.activation(self.input_layer(x))
        for hidden in self.hidden_layers:
            x = self.activation(hidden(x))
        x = self.output_layer(x)
        return x

### 各種パラメータの設定

In [ ]:
from typing import Literal
from pydantic import BaseModel
from pydantic.dataclasses import dataclass

二次元解析条件

In [ ]:
StressCondition = Literal["plane_strain", "plane_stress"]

材料定数

In [ ]:
@dataclass
class MaterialConstants:
    young: float = 2.05
    poisson: float = 0.27

ハイパーパラメータ

In [ ]:
@dataclass
class Params:
    # ネットワーク
    n_hidden_layers: int = 10
    n_hidden_neurons: int = 30
    # 解析対象
    width: float = 1.0
    height: float = 0.2
    n_collocation_samples: int = 200
    n_boundary_samples: int = 200
    use_lhs: bool = True
    # 損失関数
    a_eom: float = 1.0
    a_cl: float = 1.0
    a_nbc: float = 1.0
    a_dbc: float = 1.0
    # Optimizerの設定
    # Adam
    use_adam1: bool = True
    lr_adam1: float = 1.0e-3
    steps_adam1: int = 2000
    # Adam(2回目)
    use_adam2: bool = True
    lr_adam2: float = 1.0e-4
    steps_adam2: int = 2000
    # L-BFGS
    use_lbfgs: bool = True
    max_iter: int = 100000
    tolerance_grad: float = 1.0e-5
    tolerance_change: float = 1.0e-9
    history_size: int = 50
    line_search_fn: str = "strong_wolfe"
    # 表面力
    t0: float = -5.0e-8
    # 初期化のGain
    gain: float = 0.5
    # 二次元解析条件
    stress_condition: StressCondition = "plane_strain"

### 損失関数の設定

支配方程式に関する損失

In [ ]:
def gradient(
    dy: torch.Tensor,
    dx: list[torch.Tensor],
) -> list[torch.Tensor]:
    """勾配の計算 dy/dx^i"""
    dy_dx = torch.autograd.grad(
        [dy],
        dx,
        grad_outputs=[torch.ones_like(dy)],
        create_graph=True,
        retain_graph=True,
        allow_unused=False,
    )

    grads = [
        grad if grad is not None else torch.zeros_like(dx[i])
        for i, grad in enumerate(dy_dx)
    ]
    return grads

In [ ]:
def loss_fn_pde(
    net,
    x: torch.Tensor,
    b: torch.Tensor,
    mater: MaterialConstants,
    stress_condition: StressCondition = "plane_strain",
) -> tuple[torch.Tensor, torch.Tensor]:
    """支配方程式に関する損失の計算"""
    # x1, x2 -> u1, u2, s11, s22, s12
    x1 = x[:, 0].clone().detach().requires_grad_(True)
    x2 = x[:, 1].clone().detach().requires_grad_(True)
    X = torch.stack([x1, x2], dim=1)
    u1, u2, s11, s22, s12 = net(X).unbind(dim=1)
    b1, b2 = b.unbind(dim=1)

    # 応力と変位の空間勾配を計算
    (s11_1,) = gradient(s11, [x1])
    (s22_2,) = gradient(s22, [x2])
    s12_1, s12_2 = gradient(s12, [x1, x2])
    u1_1, u1_2 = gradient(u1, [x1, x2])
    u2_1, u2_2 = gradient(u2, [x1, x2])

    # ひずみ
    e11 = u1_1
    e22 = u2_2
    e12 = 0.5 * (u1_2 + u2_1)

    if stress_condition == "plane_strain":
        # 平面ひずみ条件
        # λ = Eν/((1+ν)(1-2ν))
        c_lambda = (
            mater.young
            * mater.poisson
            / ((1.0 + mater.poisson) * (1.0 - 2.0 * mater.poisson))
        )
        # μ = E/(2*(1+ν))
        c_mu = mater.young / (2.0 * (1.0 + mater.poisson))

        # 構成式の残差
        rc11 = c_lambda * (e11 + e22) + 2 * c_mu * e11 - s11
        rc22 = c_lambda * (e11 + e22) + 2 * c_mu * e22 - s22
        rc12 = 2 * c_mu * e12 - s12
        loss_cl = torch.mean(rc11**2 + rc22**2 + rc12**2)
    else:
        # 平面応力条件
        # 構成式の残差
        c_coef = mater.young / (1.0 - mater.poisson**2)
        rc11 = c_coef * (e11 + mater.poisson * e22) - s11
        rc22 = c_coef * (mater.poisson * e11 + e22) - s22
        rc12 = (mater.young / (1.0 + mater.poisson)) * e12 - s12
        loss_cl = torch.mean(rc11**2 + rc22**2 + rc12**2)

    # 運動方程式の残差
    rm1 = s11_1 + s12_2 + b1
    rm2 = s12_1 + s22_2 + b2
    loss_eom = torch.mean(rm1**2 + rm2**2)

    return loss_cl, loss_eom

Neumann境界条件に関する損失

In [ ]:
def loss_fn_bc_neumann(
    net, x: torch.Tensor, n: torch.Tensor, tbar: torch.Tensor
) -> torch.Tensor:
    """Neumann境界条件に関する損失を計算"""
    # x1, x2 -> u1, u2, s11, s22, s12
    _, _, s11, s22, s12 = net(x).unbind(dim=1)
    n1, n2 = n.unbind(dim=1)
    tbar1, tbar2 = tbar.unbind(dim=1)

    # 表面力ti
    t1 = s11 * n1 + s12 * n2
    t2 = s12 * n1 + s22 * n2
    # 応力から計算された表面力と境界条件として与えた表面力の平均二乗誤差
    loss_nbc = torch.mean((tbar1 - t1) ** 2 + (tbar2 - t2) ** 2)

    return loss_nbc

Dirichlet境界条件に関する損失

In [ ]:
def loss_fn_bc_dirichlet(
    net, x: torch.Tensor, ubar: torch.Tensor
) -> torch.Tensor:
    """Dirichlet境界条件に関する損失を計算"""
    # x1, x2 -> u1, u2
    u1, u2, _, _, _ = net(x).unbind(dim=1)
    ubar1, ubar2 = ubar.unbind(dim=1)
    # 出力された変位と変位境界条件の平均二乗誤差
    loss_dbc = torch.mean((ubar1 - u1) ** 2 + (ubar2 - u2) ** 2)

    return loss_dbc

### 学習用各種関数設定

解析条件をまとめたクラス

In [ ]:
class Problem(BaseModel):
    x_coll: torch.Tensor  # 領域内部の評価点
    x_nbc: torch.Tensor  # Neumann境界条件の評価点
    x_dbc: torch.Tensor  # Dirichlet境界条件の評価点
    n_nbc: torch.Tensor  # 法線ベクトル(Neumann境界条件の損失で利用)
    t_nbc: torch.Tensor  # 表面力ベクトル(Neumann境界条件の損失で利用)
    d_dbc: torch.Tensor  # 強制変位（Neumann境界条件の損失で利用）
    bf: torch.Tensor  # 物体力ベクトル

    class Config:
        arbitrary_types_allowed = True

各種損失の記録用クラス

In [ ]:
class Loss(BaseModel):
    all: list[float] = []
    constitutive: list[float] = []
    eom: list[float] = []
    neumann: list[float] = []
    dirichlet: list[float] = []

学習用関数(汎用)

In [ ]:
def train(
    net,
    optimizer,
    prob: Problem,
    mater: MaterialConstants,
    param: Params,
    losses: Loss,
    num_epochs: int,
    grad_clip_value: float = 1.0,
):
    net.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        # PDE(運動方程式，構成式)に関する損失
        loss_cl, loss_eom = loss_fn_pde(
            net,
            prob.x_coll,
            prob.bf,
            mater,
            param.stress_condition,
        )
        # Neumann B.C.に関する損失
        loss_bc_neumann = loss_fn_bc_neumann(net, prob.x_nbc, prob.n_nbc, prob.t_nbc)
        # Dirichlet B.C.に関する損失
        loss_bc_dirichlet = loss_fn_bc_dirichlet(net, prob.x_dbc, prob.d_dbc)
        # Total loss
        loss = (
            param.a_cl * loss_cl
            + param.a_eom * loss_eom
            + param.a_nbc * loss_bc_neumann
            + param.a_dbc * loss_bc_dirichlet
        )

        # 誤差逆伝播
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), grad_clip_value)
        optimizer.step()

        # 各損失の記録
        losses.all.append(loss.item())
        losses.constitutive.append(loss_cl.item())
        losses.eom.append(loss_eom.item())
        losses.neumann.append(loss_bc_neumann.item())
        losses.dirichlet.append(loss_bc_dirichlet.item())

        if epoch % 100 == 0:
            print(
                "Epoch: %d, loss_total: %.3e, loss_constitutive: %.3e, loss_EoM: %.3e, loss_Neumann: %.3e, loss_Dirichlet: %.3e"
                % (
                    epoch,
                    loss.item(),
                    loss_cl.item(),
                    loss_eom.item(),
                    loss_bc_neumann.item(),
                    loss_bc_dirichlet.item(),
                )
            )

学習用関数(L-BFGS用) < 複数回勾配の評価が必要なため別途用意

In [ ]:
def train_bfgs(
    net,
    optimizer,
    prob: Problem,
    mater: MaterialConstants,
    param: Params,
    losses: Loss,
):
    net.train()

    cnt = 0

    def closure():
        nonlocal cnt
        optimizer.zero_grad()
        # PDE(運動方程式，構成式)に関する損失
        loss_cl, loss_eom = loss_fn_pde(
            net, prob.x_coll, prob.bf, mater, param.stress_condition
        )
        # Neumann B.C.に関する損失
        loss_bc_neumann = loss_fn_bc_neumann(net, prob.x_nbc, prob.n_nbc, prob.t_nbc)
        # Dirichlet B.C.に関する損失
        loss_bc_dirichlet = loss_fn_bc_dirichlet(net, prob.x_dbc, prob.d_dbc)
        # Total loss
        loss = (
            param.a_cl * loss_cl
            + param.a_eom * loss_eom
            + param.a_nbc * loss_bc_neumann
            + param.a_dbc * loss_bc_dirichlet
        )
        # 誤差逆伝播
        loss.backward(retain_graph=True)

        # 各損失の記録
        losses.all.append(loss.item())
        losses.constitutive.append(loss_cl.item())
        losses.eom.append(loss_eom.item())
        losses.neumann.append(loss_bc_neumann.item())
        losses.dirichlet.append(loss_bc_dirichlet.item())
        cnt += 1

        if cnt % 100 == 0:
            print(
                "%d th iterations, loss_total: %.3e, loss_constitutive: %.3e, loss_EoM: %.3e, loss_Neumann: %.3e, loss_Dirichlet: %.3e"
                % (
                    cnt,
                    loss.item(),
                    loss_cl.item(),
                    loss_eom.item(),
                    loss_bc_neumann.item(),
                    loss_bc_dirichlet.item(),
                )
            )
        return loss

    optimizer.step(closure)

### 結果表示用関数

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

解析条件の表示

In [ ]:
def plot_analysis_conditions(
    *,
    x_coll: np.ndarray,
    x_nbc: np.ndarray,
    x_dbc: np.ndarray,
    output_path: str | Path | None = None,
):
    """解析条件の表示"""
    _, ax = plt.subplots()
    ax.set_aspect("equal")
    ax.scatter(
        x_coll[:, 0],
        x_coll[:, 1],
        marker="x",
        color="black",
        s=0.01,
        label="Collocation points",
    )
    ax.scatter(
        x_nbc[:, 0],
        x_nbc[:, 1],
        marker="x",
        color="red",
        s=1,
        label="Neumann B.C.",
    )
    ax.scatter(
        x_dbc[:, 0],
        x_dbc[:, 1],
        marker="x",
        color="blue",
        s=1,
        label="Dirichlet B.C.",
    )
    ax.axis("off")
    ax.legend(
        bbox_to_anchor=(1.05, 0.5),
        loc="center left",
        borderaxespad=0,
        frameon=False,
    )

    if output_path:
        plt.savefig(output_path, dpi=300, transparent=True)

各損失の表示

In [ ]:
def plot_loss(losses: Loss, output_path: str | Path | None = None):
    """各損失の表示"""
    _, ax = plt.subplots()

    # 損失のプロット
    ax.plot(losses.all, label="Total loss", linestyle="-")
    ax.plot(losses.constitutive, label="Constitutive law", linestyle="--")
    ax.plot(losses.eom, label="Equation of motion", linestyle=":")
    ax.plot(losses.neumann, label="Neumann B.C.", linestyle="-.")
    ax.plot(losses.dirichlet, label="Dirichlet B.C.", linestyle="-.")

    # ラベル設定
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss")
    ax.set_yscale("log")
    ax.legend(loc="upper right", frameon=False)

    # 軸の設定
    ax.tick_params(axis="both", direction="in", top=True, right=True)
    ax.yaxis.set_minor_locator(ticker.LogLocator(base=10.0, subs="auto", numticks=10))
    ax.tick_params(axis="y", which="both", direction="in", right=True)

    if output_path:
        plt.savefig(output_path, dpi=300, transparent=True)

解析結果の表示

In [ ]:
def plot_results(
    coords: np.ndarray,  # 座標
    vals: list[list[tuple[str, np.ndarray]]],  # (ラベル, 値)の組
    output_path: str | Path | None = None,
):
    """解析結果の表示"""
    n_rows = len(vals)
    n_cols = max([len(v) for v in vals])

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
    for irow, fields in enumerate(vals):
        for icol, (name, val) in enumerate(fields):
            ax = axs[irow, icol]
            sc = ax.scatter(
                coords[:, 0],
                coords[:, 1],
                c=val,
                alpha=0.7,
                edgecolors="none",
                cmap="rainbow",
                marker="o",
                s=2,
            )
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_aspect("equal")
            ax.set_title(name)
            fig.colorbar(sc, ax=ax)
            ax.axis("off")

        # 不要な枠削除
        for icol in range(len(fields) - n_cols, 0):
            axs[0, icol].axis("off")

    plt.tight_layout()

    if output_path:
        plt.savefig(output_path, dpi=300, transparent=True)

### 学習の実行

In [ ]:
# 材料定数
mater = MaterialConstants()
# ハイパーパラメータ
params = Params()

学習に用いる評価点の生成と境界条件の指定

In [ ]:
from pyDOE import lhs

In [ ]:
def generate_samples(
    size: tuple[float, float],
    n_collocation_samples: tuple[int, int],
    n_boundary_samples: tuple[int, int],
    t0: float,
    use_lhs: bool = True,
) -> Problem:
    """学習に用いる評価点の生成と境界条件の設定を行い，Problemクラスにまとめて返す"""
    # 解析領域の大きさ
    width, height = size
    # 領域内部の評価点数
    n_coll_samples_x, n_coll_samples_y = n_collocation_samples
    # 境界上の評価点数
    n_b_samples_x, n_b_samples_y = n_boundary_samples

    # 領域内部の評価点を生成
    if use_lhs:
        x_coll = lhs(2, n_coll_samples_x * n_coll_samples_y)
        x_coll1 = x_coll[:, 0] * width
        x_coll2 = x_coll[:, 1] * height
        x_coll = np.vstack((x_coll1, x_coll2)).T
    else:
        x_coll1 = np.linspace(0.0, width, num=n_coll_samples_x)
        x_coll2 = np.linspace(0.0, height, num=n_coll_samples_y)
        x_coll1, x_coll2 = np.meshgrid(x_coll1, x_coll2)
        x_coll = np.vstack((x_coll1.flatten(), x_coll2.flatten())).T

    # left (Dirichlet B.C.)
    if use_lhs:
        x_l1 = np.zeros(n_b_samples_y)
        x_l2 = lhs(1, samples=n_b_samples_y)[:, 0] * height
        x_left = np.vstack((x_l1, x_l2)).T
    else:
        x_left = np.zeros((n_b_samples_y, 2))
        x_left[:, 1] = np.linspace(0.0, height, num=n_b_samples_y)
    # 左辺は完全固定
    d_left = np.zeros((n_b_samples_y, 2))

    # right (Neumann B.C.)
    if use_lhs:
        x_r1 = np.ones(n_b_samples_y) * width
        x_r2 = lhs(1, samples=n_b_samples_y)[:, 0] * height
        x_right = np.vstack((x_r1, x_r2)).T
    else:
        x_right = np.zeros((n_b_samples_y, 2))
        x_right[:, 0] = width
        x_right[:, 1] = np.linspace(0.0, height, num=n_b_samples_y)
    # 法線ベクトル
    n_right = np.zeros((n_b_samples_y, 2))
    n_right[:, 0] = 1.0
    # 右辺上半分の領域に分布荷重を与える
    t_right = np.zeros((n_b_samples_y, 2))
    t_right[:, 1] = np.where(x_right[:, 1] >= height / 2, t0, 0.0)

    # bottom (Neumann B.C.)
    if use_lhs:
        x_b1 = lhs(1, samples=n_b_samples_x)[:, 0] * width
        x_b2 = np.zeros(n_b_samples_x)
        x_bottom = np.vstack((x_b1, x_b2)).T
    else:
        x_bottom = np.zeros((n_b_samples_x, 2))
        x_bottom[:, 0] = np.linspace(0.0, width, num=n_b_samples_x)
    # 法線ベクトル
    n_bottom = np.zeros((n_b_samples_x, 2))
    n_bottom[:, 1] = -1.0
    # 表面力は零
    t_bottom = np.zeros((n_b_samples_x, 2))

    # Top (Neumann B.C.)
    if use_lhs:
        x_t1 = lhs(1, samples=n_b_samples_x)[:, 0] * width
        x_t2 = np.ones(n_b_samples_x) * height
        x_top = np.vstack((x_t1, x_t2)).T
    else:
        x_top = np.zeros((n_b_samples_x, 2))
        x_top[:, 0] = np.linspace(0.0, width, num=n_b_samples_x)
        x_top[:, 1] = height
    # 法線ベクトル
    n_top = np.zeros((n_b_samples_x, 2))
    n_top[:, 1] = 1.0
    # 表面力は零
    t_top = np.zeros((n_b_samples_x, 2))

    # Dirichlet B.C.(let)の評価点，強制変位
    x_dbc = x_left
    d_dbc = d_left

    # Neumann B.C.(right, top, bottom)の評価点，法線ベクトル，表面力
    x_nbc = np.concatenate((x_right, x_top, x_bottom), axis=0)
    n_nbc = np.concatenate((n_right, n_top, n_bottom), axis=0)
    t_nbc = np.concatenate((t_right, t_top, t_bottom), axis=0)

    # 物体力(今回は零)
    bf = np.zeros(x_coll.shape)

    # np.ndarray > torch.Tensor
    x_coll = torch.tensor(
        x_coll, dtype=torch.float32, device=device, requires_grad=True
    )
    x_nbc = torch.tensor(x_nbc, dtype=torch.float32, device=device, requires_grad=True)
    n_nbc = torch.tensor(n_nbc, dtype=torch.float32, device=device, requires_grad=True)
    t_nbc = torch.tensor(t_nbc, dtype=torch.float32, device=device, requires_grad=True)
    x_dbc = torch.tensor(x_dbc, dtype=torch.float32, device=device, requires_grad=True)
    d_dbc = torch.tensor(d_dbc, dtype=torch.float32, device=device, requires_grad=True)
    bf = torch.tensor(bf, dtype=torch.float32, device=device, requires_grad=True)

    return Problem(
        x_coll=x_coll,
        x_nbc=x_nbc,
        x_dbc=x_dbc,
        n_nbc=n_nbc,
        t_nbc=t_nbc,
        d_dbc=d_dbc,
        bf=bf,
    )

問題の設定

In [ ]:
prob = generate_samples(
    size=(params.width, params.height),
    n_collocation_samples=(
        int(params.n_collocation_samples * params.width),
        int(params.n_collocation_samples * params.height),
    ),
    n_boundary_samples=(
        int(params.n_boundary_samples * params.width),
        int(params.n_boundary_samples * params.height),
    ),
    t0=params.t0,
    use_lhs=params.use_lhs,
)

評価点の確認

In [ ]:
plot_analysis_conditions(x_coll=prob.x_coll.cpu().detach().numpy(), x_nbc=prob.x_nbc.cpu().detach().numpy(), x_dbc=prob.x_dbc.cpu().detach().numpy())

ニューラルネットワークの生成

In [ ]:
net = Net(
    n_hidden_layers=params.n_hidden_layers,
    n_hidden_neurons=params.n_hidden_neurons,
    gain=params.gain,
).to(device)
print(net)

学習の実行

In [ ]:
losses = Loss()

先行研究に倣い，Adam(学習率大) > Adam(学習率小) > L-BFGSの順に実行

In [ ]:
import torch.optim as optim

In [ ]:
def tran_pinns(net, prob: Problem, mater: MaterialConstants, params: Params, losses: Loss):
    """Adam(学習率大) > Adam(学習率小) > L-BFGSの順に学習"""
    # Adam(学習率大)
    if params.use_adam1:
        optimizer = optim.Adam(net.parameters(), lr=params.lr_adam1)
        print("Start train [Adam]")
        train(
            net,
            optimizer,
            prob,
            mater,
            params,
            losses,
            params.steps_adam1,
        )
        print("End train [Adam]")

    # Adam (学習率小)
    if params.use_adam2:
        optimizer2 = optim.Adam(net.parameters(), lr=params.lr_adam2)
        print("Start train [Adam2]")
        train(
            net,
            optimizer2,
            prob,
            mater,
            params,
            losses,
            params.steps_adam2,
        )
        print("End train [Adam2]")

    # L-BFGS (これまでの学習状況によっては実行されない)
    if params.use_lbfgs:
        optimizer_lbfgs = optim.LBFGS(
            net.parameters(),
            max_iter=params.max_iter,
            tolerance_grad=params.tolerance_grad,
            tolerance_change=params.tolerance_change,
            history_size=params.history_size,
            line_search_fn=params.line_search_fn,
        )
        print("Start train [LBFGS]")
        train_bfgs(
            net,
            optimizer_lbfgs,
            prob,
            mater,
            params,
            losses,
        )
        print("End train [LBFGS]")


In [ ]:
%%time
tran_pinns(net, prob, mater, params, losses)

損失関数のプロット

In [ ]:
plot_loss(losses, output_path="loss.png")

学習後のモデルで変位と応力の分布を計算

In [ ]:
# 結果を評価するための点を均等に配置する
n_eval_points = 200
width, height = params.width, params.height
xp1 = np.linspace(0.0, width, num=int(n_eval_points * width))
xp2 = np.linspace(0.0, height, num=int(n_eval_points * height))
xp1, xp2 = np.meshgrid(xp1, xp2)
xp = np.vstack((xp1.flatten(), xp2.flatten())).T

# 学習後のモデルで変位と応力を予測する
y_preds = net(torch.tensor(xp, dtype=torch.float32).to(device)).unbind(dim=1)
y_preds = [y.cpu().detach().numpy() for y in y_preds]
u_pred, v_pred, s11_pred, s22_pred, s12_pred = y_preds

# 変位を1000倍で表示
x_pred = xp + 1000 * np.column_stack([u_pred, v_pred])
plot_results(
    coords=x_pred,
    vals=[
        [(r"$u$", u_pred), (r"$v$", v_pred)],
        [
            (r"$\sigma^{11}$", s11_pred),
            (r"$\sigma^{22}$", s22_pred),
            (r"$\sigma^{12}$", s12_pred),
        ],
    ],
    output_path="results.png",
)

## 補足

### 解析例①

- [C. Rao et al., J. Eng. Mech. (2021)](https://ascelibrary.org/doi/abs/10.1061/%28ASCE%29EM.1943-7889.0001947)([arXiv版](https://arxiv.org/abs/2006.08472))のAppendix B記載の解析に類似の解析
- ヤング率，ポアソン比は論文に記載の$E=10$，$\nu =0.2$を用いる
- 評価点数の設定は論文の条件に倣う
- 隠れ層の数，ニューロン数は論文に記載の構成だとうまく学習が進まなかったので変更している
- 論文にはこの解析に対するオプティマイザーの記載がないため，論文中の他の解析で使用されているAdam(学習率$10^{-3}$) 2,000エポック，Adam(学習率$10^{-4}$) 2,000エポック，L-BFGSの順で行う

In [ ]:
# 材料定数
mater_rao = MaterialConstants(young=10.0, poisson=0.2)
# ハイパーパラメータ
params_rao = Params(
    width=1.0,
    height=1.0,
    n_hidden_layers=10,
    n_hidden_neurons=40,
    n_collocation_samples=100,
    n_boundary_samples=100,
    use_adam1=True,
    lr_adam1=1.0e-3,
    steps_adam1=2000,
    use_adam2=True,
    lr_adam2=1.0e-4,
    steps_adam2=2000,
)

問題設定

In [ ]:
def generate_samples_rao(
    size: tuple[float, float],
    n_collocation_samples: tuple[int, int],
    n_boundary_samples: tuple[int, int],
    use_lhs: bool = True,
) -> Problem:
    """学習に用いる評価点の生成と境界条件の設定を行い，Problemクラスにまとめて返す"""
    # 解析領域の大きさ
    width, height = size
    # 領域内部の評価点数
    n_coll_samples_x, n_coll_samples_y = n_collocation_samples
    # 境界上の評価点数
    n_b_samples_x, n_b_samples_y = n_boundary_samples

    # 領域内部の評価点を生成
    if use_lhs:
        x_coll = lhs(2, n_coll_samples_x * n_coll_samples_y)
        x_coll1 = x_coll[:, 0] * width
        x_coll2 = x_coll[:, 1] * height
        x_coll = np.vstack((x_coll1, x_coll2)).T
    else:
        x_coll1 = np.linspace(0.0, width, num=n_coll_samples_x)
        x_coll2 = np.linspace(0.0, height, num=n_coll_samples_y)
        x_coll1, x_coll2 = np.meshgrid(x_coll1, x_coll2)
        x_coll = np.vstack((x_coll1.flatten(), x_coll2.flatten())).T
    # Rao et al., の個数に合わせる
    x_coll = x_coll[
        : (n_coll_samples_x * n_coll_samples_y - 2 * (n_b_samples_x + n_b_samples_y))
    ]

    # left (Neumann B.C.)
    if use_lhs:
        x_l1 = np.zeros(n_b_samples_y)
        x_l2 = lhs(1, samples=n_b_samples_y)[:, 0] * height
        x_left = np.vstack((x_l1, x_l2)).T
    else:
        x_left = np.zeros((n_b_samples_y, 2))
        x_left[:, 1] = np.linspace(0.0, height, num=n_b_samples_y)
    # 法線ベクトル
    n_left = np.zeros((n_b_samples_y, 2))
    n_left[:, 0] = -1.0
    # 表面力は零
    t_left = np.zeros((n_b_samples_y, 2))

    # right (Neumann B.C.)
    if use_lhs:
        x_r1 = np.ones(n_b_samples_y) * width
        x_r2 = lhs(1, samples=n_b_samples_y)[:, 0] * height
        x_right = np.vstack((x_r1, x_r2)).T
    else:
        x_right = np.zeros((n_b_samples_y, 2))
        x_right[:, 0] = width
        x_right[:, 1] = np.linspace(0.0, height, num=n_b_samples_y)
    # 法線ベクトル
    n_right = np.zeros((n_b_samples_y, 2))
    n_right[:, 0] = 1.0
    # 表面力は零
    t_right = np.zeros((n_b_samples_y, 2))

    # bottom (Dirichlet B.C.)
    if use_lhs:
        x_b1 = lhs(1, samples=n_b_samples_x)[:, 0] * width
        x_b2 = np.zeros(n_b_samples_x)
        x_bottom = np.vstack((x_b1, x_b2)).T
    else:
        x_bottom = np.zeros((n_b_samples_x, 2))
        x_bottom[:, 0] = np.linspace(0.0, width, num=n_b_samples_x)
    d_bottom = np.zeros((n_b_samples_x, 2))

    # Top left (Dirichlet B.C.)
    if use_lhs:
        x_tl1 = lhs(1, samples=int(n_b_samples_x / 2))[:, 0] * width / 2
        x_tl2 = np.ones(int(n_b_samples_x / 2)) * height
        x_tl = np.vstack((x_tl1, x_tl2)).T
    else:
        x_tl = np.zeros((int(n_b_samples_x / 2), 2))
        x_tl[:, 0] = np.linspace(
            0.0, width / 2, num=int(n_b_samples_x / 2)
        )
        x_tl[:, 1] = height
    d_top = np.zeros((int(n_b_samples_x / 2), 2))
    d_top[:, 1] = 0.1  # y方向に0.1の強制変位

    # Top right (Neumann B.C.)
    if use_lhs:
        x_tr1 = lhs(1, samples=int(n_b_samples_x / 2))[:, 0] * width / 2 + width / 2
        x_tr2 = np.ones(int(n_b_samples_x / 2)) * height
        x_tr = np.vstack((x_tr1, x_tr2)).T
    else:
        x_tr = np.zeros((int(n_b_samples_x / 2), 2))
        x_tr[:, 0] = np.linspace(
            width / 2, width, num=int(n_b_samples_x / 2)
        )
        x_tr[:, 1] = height
    # 法線ベクトル
    n_top = np.zeros((int(n_b_samples_x / 2), 2))
    n_top[:, 1] = 1.0
    # 表面力は零
    t_top = np.zeros((int(n_b_samples_y / 2), 2))

    # Dirichlet B.C.(bottom, top left)の評価点，強制変位
    x_dbc = np.concatenate((x_bottom, x_tl), axis=0)
    d_dbc = np.concatenate((d_bottom, d_top), axis=0)

    # Neumann B.C.(left, right, top right)の評価点，法線ベクトル，表面力
    x_nbc = np.concatenate((x_left, x_right, x_tr), axis=0)
    n_nbc = np.concatenate((n_left, n_right, n_top), axis=0)
    t_nbc = np.concatenate((t_left, t_right, t_top), axis=0)

    # 物体力(今回は零)
    bf = np.zeros(x_coll.shape)

    # np.ndarray > torch.Tensor
    x_coll = torch.tensor(
        x_coll, dtype=torch.float32, device=device, requires_grad=True
    )
    x_nbc = torch.tensor(x_nbc, dtype=torch.float32, device=device, requires_grad=True)
    n_nbc = torch.tensor(n_nbc, dtype=torch.float32, device=device, requires_grad=True)
    t_nbc = torch.tensor(t_nbc, dtype=torch.float32, device=device, requires_grad=True)
    x_dbc = torch.tensor(x_dbc, dtype=torch.float32, device=device, requires_grad=True)
    d_dbc = torch.tensor(d_dbc, dtype=torch.float32, device=device, requires_grad=True)
    bf = torch.tensor(bf, dtype=torch.float32, device=device, requires_grad=True)

    return Problem(
        x_coll=x_coll,
        x_nbc=x_nbc,
        x_dbc=x_dbc,
        n_nbc=n_nbc,
        t_nbc=t_nbc,
        d_dbc=d_dbc,
        bf=bf,
    )

In [ ]:
prob_rao = generate_samples_rao(
    size=(params_rao.width, params_rao.height),
    n_collocation_samples=(
        int(params_rao.n_collocation_samples * params_rao.width),
        int(params_rao.n_collocation_samples * params_rao.height),
    ),
    n_boundary_samples=(
        int(params_rao.n_boundary_samples * params_rao.width),
        int(params_rao.n_boundary_samples * params_rao.height),
    ),
    use_lhs=params_rao.use_lhs,
)

In [ ]:
plot_analysis_conditions(x_coll=prob_rao.x_coll.cpu().detach().numpy(), x_nbc=prob_rao.x_nbc.cpu().detach().numpy(), x_dbc=prob_rao.x_dbc.cpu().detach().numpy())

学習実行

In [ ]:
net_rao = Net(
    n_hidden_layers=params_rao.n_hidden_layers,
    n_hidden_neurons=params_rao.n_hidden_neurons,
    gain=params_rao.gain,
).to(device)
print(net_rao)

In [ ]:
losses_rao = Loss()

In [ ]:
%%time
tran_pinns(net_rao, prob_rao, mater_rao, params_rao, losses_rao)

結果表示

In [ ]:
plot_loss(losses_rao, output_path="loss_rao.png")

In [ ]:
# 結果を評価するための点を均等に配置する
n_eval_points = 200
width, height = params_rao.width, params_rao.height
xp1 = np.linspace(0.0, width, num=int(n_eval_points * width))
xp2 = np.linspace(0.0, height, num=int(n_eval_points * height))
xp1, xp2 = np.meshgrid(xp1, xp2)
xp = np.vstack((xp1.flatten(), xp2.flatten())).T

# 学習後のモデルで変位と応力を予測する
y_preds = net_rao(torch.tensor(xp, dtype=torch.float32).to(device)).unbind(dim=1)
y_preds = [y.cpu().detach().numpy() for y in y_preds]
u_pred, v_pred, s11_pred, s22_pred, s12_pred = y_preds
plot_results(
    coords=xp,
    vals=[
        [(r"$u$", u_pred), (r"$v$", v_pred)],
        [
            (r"$\sigma^{11}$", s11_pred),
            (r"$\sigma^{22}$", s22_pred),
            (r"$\sigma^{12}$", s12_pred),
        ],
    ],
    output_path="results_rao.png",
)

### 解析例②

- 単純せん断
- [GitHub: ericstewart36/pinnsforsolids](https://github.com/ericstewart36/pinnsforsolids/blob/05c37b3df15dc123db9440b32c5de63b8cfaf2a4/Stewart_Eric_final_report.pdf)の3章に記載の解析に類似の解析
  - オプティマイザーの設定はこれに倣っている
  - 評価点数は参照先と同じだが，ここではサンプリング方法にLHSを用いている
  - 物性値，ネットワークの層数は異なる値を設定している


In [ ]:
# 材料定数
mater_ss = MaterialConstants()
# ハイパーパラメータ
params_ss = Params(
    width=1.0,
    height=1.0,
    n_hidden_layers=10,
    n_hidden_neurons=40,
)

問題設定

In [ ]:
def generate_samples_ss(
    size: tuple[float, float],
    n_collocation_samples: tuple[int, int],
    n_boundary_samples: tuple[int, int],
    use_lhs: bool = True,
) -> Problem:
    """学習に用いる評価点の生成と境界条件の設定を行い，Problemクラスにまとめて返す"""
    # 解析領域の大きさ
    width, height = size
    # 領域内部の評価点数
    n_coll_samples_x, n_coll_samples_y = n_collocation_samples
    # 境界上の評価点数
    n_b_samples_x, n_b_samples_y = n_boundary_samples

    # 領域内部の評価点を生成
    if use_lhs:
        x_coll = lhs(2, n_coll_samples_x * n_coll_samples_y)
        x_coll1 = x_coll[:, 0] * width
        x_coll2 = x_coll[:, 1] * height
        x_coll = np.vstack((x_coll1, x_coll2)).T
    else:
        x_coll1 = np.linspace(0.0, width, num=n_coll_samples_x)
        x_coll2 = np.linspace(0.0, height, num=n_coll_samples_y)
        x_coll1, x_coll2 = np.meshgrid(x_coll1, x_coll2)
        x_coll = np.vstack((x_coll1.flatten(), x_coll2.flatten())).T

    # left (Neumann B.C.)
    if use_lhs:
        x_l1 = np.zeros(n_b_samples_y)
        x_l2 = lhs(1, samples=n_b_samples_y)[:, 0] * height
        x_left = np.vstack((x_l1, x_l2)).T
    else:
        x_left = np.zeros((n_b_samples_y, 2))
        x_left[:, 1] = np.linspace(0.0, height, num=n_b_samples_y)
    # 法線ベクトル
    n_left = np.zeros((n_b_samples_y, 2))
    n_left[:, 0] = -1.0
    # 表面力
    t_left = np.zeros((n_b_samples_y, 2))

    # right (Neumann B.C.)
    if use_lhs:
        x_r1 = np.ones(n_b_samples_y) * width
        x_r2 = lhs(1, samples=n_b_samples_y)[:, 0] * height
        x_right = np.vstack((x_r1, x_r2)).T
    else:
        x_right = np.zeros((n_b_samples_y, 2))
        x_right[:, 0] = width
        x_right[:, 1] = np.linspace(0.0, height, num=n_b_samples_y)
    # 法線ベクトル
    n_right = np.zeros((n_b_samples_y, 2))
    n_right[:, 0] = 1.0
    # 表面力は零
    t_right = np.zeros((n_b_samples_y, 2))

    # bottom (Dirichlet B.C.)
    if use_lhs:
        x_b1 = lhs(1, samples=n_b_samples_x)[:, 0] * width
        x_b2 = np.zeros(n_b_samples_x)
        x_bottom = np.vstack((x_b1, x_b2)).T
    else:
        x_bottom = np.zeros((n_b_samples_x, 2))
        x_bottom[:, 0] = np.linspace(0.0, width, num=n_b_samples_x)
    d_bottom = np.zeros((n_b_samples_x, 2))

    # top (Dirichlet B.C.)
    if use_lhs:
        x_t1 = lhs(1, samples=n_b_samples_x)[:, 0] * width
        x_t2 = np.ones(n_b_samples_x) * height
        x_top = np.vstack((x_t1, x_t2)).T
    else:
        x_top = np.zeros((n_b_samples_x, 2))
        x_top[:, 0] = np.linspace(0.0, width, num=n_b_samples_x)  # x = 0 ~ +1
        x_top[:, 1] = height
    d_top = np.zeros((n_b_samples_x, 2))
    # 強制変位
    d_top[:, 0] = 0.1

    # Dirichlet B.C.(bottom, top)の評価点，強制変位
    x_dbc = np.concatenate((x_bottom, x_top), axis=0)
    d_dbc = np.concatenate((d_bottom, d_top), axis=0)

    # Neumann B.C.(left, right)の評価点，法線ベクトル，表面力
    x_nbc = np.concatenate((x_left, x_right), axis=0)
    n_nbc = np.concatenate((n_left, n_right), axis=0)
    t_nbc = np.concatenate((t_left, t_right), axis=0)

    # 物体力(今回は零)
    bf = np.zeros(x_coll.shape)

    # np.ndarray > torch.Tensor
    x_coll = torch.tensor(
        x_coll, dtype=torch.float32, device=device, requires_grad=True
    )
    x_nbc = torch.tensor(x_nbc, dtype=torch.float32, device=device, requires_grad=True)
    n_nbc = torch.tensor(n_nbc, dtype=torch.float32, device=device, requires_grad=True)
    t_nbc = torch.tensor(t_nbc, dtype=torch.float32, device=device, requires_grad=True)
    x_dbc = torch.tensor(x_dbc, dtype=torch.float32, device=device, requires_grad=True)
    d_dbc = torch.tensor(d_dbc, dtype=torch.float32, device=device, requires_grad=True)
    bf = torch.tensor(bf, dtype=torch.float32, device=device, requires_grad=True)

    return Problem(
        x_coll=x_coll,
        x_nbc=x_nbc,
        x_dbc=x_dbc,
        n_nbc=n_nbc,
        t_nbc=t_nbc,
        d_dbc=d_dbc,
        bf=bf,
    )

In [ ]:
prob_ss = generate_samples_ss(
    size=(params_ss.width, params_ss.height),
    n_collocation_samples=(
        int(params_ss.n_collocation_samples * params_ss.width),
        int(params_ss.n_collocation_samples * params_ss.height),
    ),
    n_boundary_samples=(
        int(params_ss.n_boundary_samples * params_ss.width),
        int(params_ss.n_boundary_samples * params_ss.height),
    ),
    use_lhs=params_ss.use_lhs,
)

In [ ]:
plot_analysis_conditions(x_coll=prob_ss.x_coll.cpu().detach().numpy(), x_nbc=prob_ss.x_nbc.cpu().detach().numpy(), x_dbc=prob_ss.x_dbc.cpu().detach().numpy())

学習実行

In [ ]:
net_ss = Net(
    n_hidden_layers=params_ss.n_hidden_layers,
    n_hidden_neurons=params_ss.n_hidden_neurons,
    gain=params_ss.gain,
).to(device)
print(net_ss)

In [ ]:
losses_ss = Loss()

In [ ]:
%%time
tran_pinns(net_ss, prob_ss, mater_ss, params_ss, losses_ss)

結果表示

In [ ]:
plot_loss(losses_ss, output_path="loss_ss.png")

In [ ]:
# 結果を評価するための点を均等に配置する
n_eval_points = 200
width, height = params_ss.width, params_ss.height
xp1 = np.linspace(0.0, width, num=int(n_eval_points * width))
xp2 = np.linspace(0.0, height, num=int(n_eval_points * height))
xp1, xp2 = np.meshgrid(xp1, xp2)
xp = np.vstack((xp1.flatten(), xp2.flatten())).T

# 学習後のモデルで変位と応力を予測する
y_preds = net_ss(torch.tensor(xp, dtype=torch.float32).to(device)).unbind(dim=1)
y_preds = [y.cpu().detach().numpy() for y in y_preds]
u_pred, v_pred, s11_pred, s22_pred, s12_pred = y_preds
x_pred = xp + np.column_stack([u_pred, v_pred])
plot_results(
    coords=x_pred,
    vals=[
        [(r"$u$", u_pred), (r"$v$", v_pred)],
        [
            (r"$\sigma^{11}$", s11_pred),
            (r"$\sigma^{22}$", s22_pred),
            (r"$\sigma^{12}$", s12_pred),
        ],
    ],
    output_path="results_ss.png",
)

以上